In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

### Request
- get request és átlakítás

In [2]:
city = "I.%20ker%C3%BClet"

response = requests.get(f"https://data2.openstreetmap.hu/addrtel.php?addr={city}")
text = response.text
soup = BeautifulSoup(text, 'html.parser')

### Tartalom
- a táblázat megkeresése és td tagekben található szövegek kiválogatása

In [3]:
table = soup.find('table')

header = []
rows = []
for i, row in enumerate(table.find_all('tr')):
    if i == 0:
        header = [th.text.strip() for th in row.find_all('th')]
    else:
        rows.append([td.text.strip() for td in row.find_all('td')])
print(header)

['addr:postcode', 'addr:city', 'addr:street', 'addr:housenumber', 'name', 'Térkép', '', 'OSM id (JOSM link)']


### DataFrame létrehozása

In [4]:
content = []
for i, row in enumerate(rows):    
    content.append(row[:4])

In [5]:
df = pd.DataFrame(content, columns=['POSTCODE', 'CITY', 'STREET', 'NUMBER'])

In [7]:
df

,POSTCODE,CITY,STREET,NUMBER
0,1011,Budapest,Apor Péter utca,1
1,1011,Budapest,Apor Péter utca,2-4
2,1011,Budapest,Apor Péter utca,3
3,1011,Budapest,Aranyhal utca,4
4,1011,Budapest,Batthyány tér,3
...,...,...,...,...
1917,,Budapest,Batthyány tér,
1918,,Budapest,Batthyány tér,
1919,,Budapest,Batthyány tér,
1920,,Budapest,Dózsa György tér,


### Tisztítás
- azok a sorok kitörlése, ahol vannak üres cellák

In [8]:
df = df[df['STREET'].str.len() > 1]
df = df[df['CITY'].str.len() > 1]
df = df[df['NUMBER'].str.len() > 1]
df = df[df['POSTCODE'].str.len() > 1]

In [9]:
df

,POSTCODE,CITY,STREET,NUMBER
1,1011,Budapest,Apor Péter utca,2-4
6,1011,Budapest,Batthyány tér,5-6
7,1011,Budapest,Batthyány tér,5-6
8,1011,Budapest,Batthyány tér,5-6
23,1011,Budapest,Bem rakpart,11
...,...,...,...,...
1877,1016,Budapest,Tigris utca,64
1892,1016,Budapest,Zsolt utca,10b
1893,1016,Budapest,Zsolt utca,11
1900,1250,Budapest,Dísz tér,15


### Házszám és utca összefűzése

In [10]:
df['ADDRESS'] = df['STREET'] + ' ' + df['NUMBER']
df.drop(['STREET', 'NUMBER'], axis=1, inplace=True)

In [11]:
df

,POSTCODE,CITY,ADDRESS
1,1011,Budapest,Apor Péter utca 2-4
6,1011,Budapest,Batthyány tér 5-6
7,1011,Budapest,Batthyány tér 5-6
8,1011,Budapest,Batthyány tér 5-6
23,1011,Budapest,Bem rakpart 11
...,...,...,...
1877,1016,Budapest,Tigris utca 64
1892,1016,Budapest,Zsolt utca 10b
1893,1016,Budapest,Zsolt utca 11
1900,1250,Budapest,Dísz tér 15
